In [1]:
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from pandas import json_normalize
import ast
nltk.download('vader_lexicon')
import pyarrow

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


User Reviews

In [2]:
#Ruta del archivo JSON
data_list = []
file_path = 'australian_user_reviews.json'

#Abrir el archivo y procesar cada línea
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

#Crear un DataFrame a partir de la lista de diccionarios
userReviewsDf = pd.DataFrame(data_list)

In [15]:
userReviewsDf=userReviewsDf.dropna(how='all')
userReviewsDf.to_parquet('data/userReviews.parquet',index=False)

In [16]:
userReviewsDf

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [4]:
userReviewsExplodedDf=userReviewsDf.explode('reviews').reset_index(drop=True)

In [5]:
userReviewsExplodedDf['review']=userReviewsExplodedDf['reviews'].apply(lambda x: x.get('review') if isinstance(x, dict) and 'review' in x else None)
userReviewsExplodedDf=userReviewsExplodedDf.dropna(subset='review')
userReviewsExplodedDf['recommend']=userReviewsExplodedDf['reviews'].apply(lambda x: x.get('recommend') if isinstance(x, dict) and 'recommend' in x else None)
userReviewsExplodedDf['item_id']=userReviewsExplodedDf['reviews'].apply(lambda x: x.get('item_id') if isinstance(x, dict) and 'item_id' in x else None)
userReviewsExplodedDf['review_date']=userReviewsExplodedDf['reviews'].apply(lambda x: x.get('posted') if isinstance(x, dict) and 'posted' in x else None)

In [7]:
userReviewsExplodedDf['review_date'] = userReviewsExplodedDf['review_date'].str.replace('Posted ', '', regex=False)
userReviewsExplodedDf['review_date']=pd.to_datetime(userReviewsExplodedDf['review_date'],format='mixed',errors='coerce')
userReviewsExplodedDf['review_year']=userReviewsExplodedDf['review_date'].dt.year
userReviewsExplodedDf['review_year']=pd.to_numeric(userReviewsExplodedDf['review_year'], errors='coerce').astype('Int64')

In [8]:
def analyze_sentiment(text):
    if text is None:
        return '1'

    sia = SentimentIntensityAnalyzer()
    sentiment_score = sia.polarity_scores(text)['compound']

    if sentiment_score >= 0.05:
        return '2'
    elif sentiment_score <= -0.05:
        return '0'
    else:
        return '1'

In [9]:
userReviewsExplodedDf['sentiment_analysis']=userReviewsExplodedDf['review'].apply(analyze_sentiment)

In [10]:
del userReviewsExplodedDf['reviews']
del userReviewsExplodedDf['review']
del userReviewsExplodedDf['user_url']

In [11]:
userReviewsExplodedDf.to_parquet('data/userReviewsExploded.parquet',index=False)

In [12]:
userReviewsExplodedDf.sort_values(by='sentiment_analysis')

,user_id,recommend,item_id,review_date,review_year,sentiment_analysis
40718,76561198068130802,False,304930,2015-07-05,2015,0
54690,76561198081924175,False,49520,2014-07-25,2014,0
54689,76561198081923658,True,570,2014-09-09,2014,0
46529,76561198031877091,True,252490,2014-01-12,2014,0
27547,CowOnWeed,True,220,2015-04-16,2015,0
...,...,...,...,...,...,...
24196,ACaesarSalad,True,339800,2015-06-22,2015,2
24197,ACaesarSalad,True,39000,2014-05-25,2014,2
24198,rylo20340,True,238960,2013-10-26,2013,2
24167,K33mstar,True,48000,2014-09-03,2014,2
